# 12. Preprocesado tras incorporar ASAS-SN v band al dataset

In [1]:
# Configuración general para evitar errores de warnings y compatibilidad
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["RICH_NO_RICH"] = "1"
print("Configuración de entorno aplicada.")

Configuración de entorno aplicada.


Dentro del script, este es el preprocesado que se hace por cada curva:

| Paso                                   | ¿Incluido?    | Descripción                                                               |
| -------------------------------------- | ------------- | ------------------------------------------------------------------------- |
| Orden temporal                         | ❌ (implícito) | No se fuerza explícitamente `sort_values('tiempo')`, pero puede añadirse. |
| Sustitución de `NaN`                   | ✅             | Usa `np.nan_to_num` con la mediana.                                       |
| Filtro por longitud mínima             | ✅             | `if len(magnitudes) < MIN_POINTS`                                         |
| Filtro por dispersión mínima (`std`)   | ✅             | `if np.std(magnitudes) < MIN_STD`                                         |
| Normalización robusta (mediana/IQR)    | ✅             | `(magnitudes - median) / iqr`                                             |
| Clip de valores extremos               | ✅             | `np.clip(..., -1000, 1000)`                                               |
| Padding y atención mask                | ✅             | Rellena hasta `seq_length`, y genera `attention_mask`                     |
| Validación de `nan`/`inf` tras normal. | ✅             | Verifica si hay valores no válidos después de normalizar.                 |
| Normalización de clase                 | ✅             | Aplica `normalize_label` a la clase de entrada.                           |

¿Qué se podría mejorar?

1. **Orden temporal explícito**:
   Actualmente **no se aplica `sort_values("tiempo")`** sobre cada curva antes de procesarla. Aunque muchas curvas ya vienen ordenadas, sería más robusto añadir:

   ```python
   df = df.sort_values(by=find_column(df.columns, "tiempo"))
   ```

   ...como primer paso dentro de `process_single_curve()`.

2. **Soporte para features adicionales**:
   Ahora solo se usa `mag` (magnitud), pero si en el futuro deseas usar también `flux`, `mag_err`, etc., habría que adaptar esta función o añadir variantes.

Como está ahora, los **únicos datos de entrada al modelo** son:

* `magnitudes_norm`: vector de `seq_length` elementos (curva preprocesada)
* `attention_mask`: vector binario que indica datos válidos
* `clase`: etiqueta de clase codificada (para entrenamiento o evaluación)

Esto es **lo correcto** para un modelo tipo Transformer que espera curvas de magnitud como secuencia 1D.



In [ ]:
from src.fase2.script_1_transformer_preprocessing_optimizado_2 import main

max_per_class_override={
    "Irregular": 9000,
    "Rotational": 9000,
    "Eclipsing Binary": 9000,
    "Delta Scuti": None,               # 7.550 → TODAS
    "RR Lyrae": None,                  # 41.208 → TODAS
    "Young Stellar Object": None,      # 9.809 → TODAS
    "Cataclysmic": None,               # 2.080 → TODAS
    "White Dwarf": None,               # 118 → TODAS
    "Variable": 1000                   # limitada por ser genérica
}

main(
    seq_length=25000,
    max_per_class=None, # usamos override completo
    max_per_class_override=max_per_class_override
)

📂 Cargando datos en lotes con PyArrow...


Agrupando curvas por objeto: 274742batch [8:11:35,  7.73batch/s]